In [ ]:
! pip install pytelegrambotapi

In [ ]:
!pip install pandas pandasai==0.8.4

In [ ]:
!pip install googletrans==4.0.0-rc1

In [2]:
import telebot
import pandas as pd
from pandasai.llm.starcoder import Starcoder
from pandasai import PandasAI
import numpy

from googletrans import Translator
import locale

In [3]:
API_TOKEN = ''
bot = telebot.TeleBot(API_TOKEN)

In [4]:
llm = Starcoder(api_token="")
df = pd.read_csv("https://raw.githubusercontent.com/noo-e/Olimpia_bot/main/Medals.csv")
pandas_ai = PandasAI(llm, conversational=False)

In [8]:
translator = Translator()
#locale.setlocale(locale.LC_COLLATE, 'pt_BR.utf-8')

In [9]:
@bot.message_handler(commands=["start"])
def send_welcome(message):
  chat_id = message.chat.id
  response = '''
Olá, seja bem-vindo(a)!
Eu sou a Olímpia, bot de informação sobre os Jogos Olímpicos de Verão Tóquio 2020.
Minhas funções são focadas no quadro de medalhas. Me peça informações sobre as olimpíadas!

Exemplos:
"Número de medalhas de ouro do Brasil"
"Número de medalhas totais do Japão"
"top 5 países que mais ganharam medalhas de ouro"

Comandos:
/paises - Lista todos os países que participaram da competição.
(Obs: Consulte como o nome do país está registrado para maiores chances de resposta.)'''
  bot.send_message(chat_id, response)

In [10]:
@bot.message_handler(commands=["paises"])
def list_countries(message):
    countries = df["Country name"].tolist()
    string_countries = '\n'.join(countries)

    translated_countries = translator.translate(string_countries, dest='pt').text
    translated_countries_list = translated_countries.splitlines()
    print(translated_countries_list)
    sorted_translated_countries_list = sorted(translated_countries_list)
    string_translated_countries = '\n'.join(sorted_translated_countries_list)
    response = f"Lista de países:\n{string_translated_countries}"
    bot.reply_to(message, response)

In [11]:
def format_response(translated_message, response):
    if isinstance(response, numpy.ndarray):
        response = '\n'.join(response)
        translated_response = translator.translate(response, dest='pt').text
        return translated_response
    elif isinstance(response, pd.DataFrame):
        formatted_message = translated_message + '\n\n'
        for index, row in response.iterrows():
            country_info = f"{index + 1}. {row['Country name']} - 🥇{row['Gold medals']} 🥈{row['Silver medals']} 🥉{row['Bronze medals']} Total: {row['Total medals']}"
            formatted_message += country_info + '\n'

        translated_response = translator.translate(formatted_message, dest='pt').text
        return translated_response

    elif isinstance(response, numpy.int64):
        if 'gold' in translated_message.lower():
            response = f"🥇 {response} 🥇"
        elif 'silver' in translated_message.lower():
            response = f"🥈 {response} 🥈"
        elif 'bronze' in translated_message.lower():
            response = f"🥉 {response} 🥉"
        elif 'total' in translated_message.lower():
            response = f"🥇🥈🥉 {response} 🥇🥈🥉"

    else:
        if "No code found in the response" in response:
            response = f"Ocorreu um erro ao processar sua pergunta. Erro: No code found in the response"

    return response

In [12]:
# Mensagens
@bot.message_handler(func=lambda message: True)
def response(message):
    translated_message = translator.translate(message.text, dest='en').text
    print(translated_message)
    response = pandas_ai.run(df, prompt=translated_message)
    #print("response: ", response)
    #print(type(response))

    formated_response = format_response(translated_message, response)
    bot.reply_to(message, formated_response)

In [ ]:
bot.polling()